In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd


import sys
sys.path.append("../../")
import os
from src.functional import free_gpu_cache
from src.utils import env_utils

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.models import load_LM
import torch
from nnsight import LanguageModel
from src.utils import env_utils

# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

lm = load_LM(
    model_key=model_key,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 30/30 [01:11<00:00,  2.38s/it]

loaded /home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-70B-Instruct | size: 36650.535 MB


In [31]:
from src.dataset import DatasetV2, SampleV2

with open (os.path.join(env_utils.DEFAULT_DATA_DIR, "dataset_v2.json"), "r") as f:
    dataset_dict = json.load(f)

dataset = DatasetV2.from_dict(dataset_dict)
print(len(dataset))

5561


In [32]:
from src.functional import get_hs, find_token_range, prepare_input, logit_lens
def collect_actor_latent_in_question(
    lm: LanguageModel,
    question: str,
    actor: str,
    layers: list = list(range(7, 28)),
    layer_name_format: str = "model.layers.{}"
) -> dict[int: torch.Tensor]:
    inputs = prepare_input(prompts=question, tokenizer=lm, return_offsets_mapping=True)
    action_last_range = find_token_range(
        string=question,
        substring=actor,
        occurrence=-1,
        offset_mapping=inputs["offset_mapping"][0],
    )
    actor_last_idx = action_last_range[1] - 1
    inputs.pop("offset_mapping")
    print(actor_last_idx, " >> ", lm.tokenizer.decode(inputs["input_ids"][0][actor_last_idx]))

    
    last_loc = (layer_name_format.format(lm.config.num_hidden_layers-1), -1)
    locations = [(layer_name_format.format(i), actor_last_idx) for i in layers] + [last_loc]

    hs = get_hs(
        lm=lm,
        input=inputs,
        locations=locations,
        return_dict=True
    )

    predicted_ans = logit_lens(lm = lm, h = hs[last_loc], k = 2)[0]
    print(f"{predicted_ans=}")

    return hs

In [33]:
idx = 25
question, answer = dataset.__getitem__(idx, set_ans="yes")
# question = question.replace("Answer:", "Answer (yes/no):")
actor = dataset.samples[idx].actor
print(question)
print(answer)

hs = collect_actor_latent_in_question(
    lm=lm,
    question=question,
    actor=actor,
    layers=list(range(7, 28)),
)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer (yes/no):" tag.

Story: Rob works in a busy restaurant. A customer asks Rob for aquavit. Rob grabs an opaque pitcher and fills it with aquavit. A coworker, who didn't hear the customer's request, swaps the aquavit in the pitcher with ouzo while Rob is attending to another task. Rob does not see the coworker swapping the contents in the pitcher and also can't see what is in it without opening it.

Question: Does Rob believe that there is aquavit in the pitcher?
Answer:
yes
147  >>   Rob
predicted_ans=PredictedToken(token=' yes', prob=0.5722293257713318, logit=21.953125, token_id=10035)


In [35]:
(6000 * 5)/60

500.0